## Recipe Builder Actions Overview

### Saving a File Cell
If you wish to save the contents of a cell, simply run it.  The `%%writefile` command at the top of the cell will write the contents of the cell to the file named at the top of the cell. You should run the cells manually when applicable. However, **pressing any of the actions at the top will automatically run all file cells relevant to the action**.

### Training and Scoring
Press the associated buttons at the top in order to run training or scoring. The training output will be shown below the `evaluator.py` cell and scoring output will be shown below the `datasaver.py` cell. You must run training at least once before you can run scoring. You may delete the output cell(s). Running training the first time or after changing `requirements.txt` will be slower since the dependencies for the recipe need to be installed, but subsequent runs will be significantly faster. If you wish to see the hidden output add `debug` to the end of the output cell and re-run it.

### Creating the Recipe
When you are done editing the recipe and satisfied with the training/scoring output, you can create a recipe from the notebook by pressing `Create Recipe`. You must run scoring at least once before you can create the recipe. After pressing it, you will see a progress bar showing how much time is left for the build to finish. If the recipe creation is successful the progress bar will be replaced by an external link that you can click to navigate to the created recipe.


## Caution!
* **Do not delete any of the file cells**
* **Do not edit the `%%writefile` line at the top of the file cells**

---

#### **Requirements File** (Optional)
Add additional libraries you wish to use in the recipe to the cell below. You can specify the version number if necessary. The file cell below is a **commented out example**.  The file structure is yaml. It adheres to the specification outlined in the [Conda Documentation](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-file-manually). **NOTE:** The name field is not allowed to be overridden in the file.

In [19]:
%%writefile ~/my-workspace/.recipes/recipe-Ru_oljS6g/requirements.txt

#dependencies:
## Conda dependencies should be attempted first
#- statsmodels=0.10.2
#- pip
#- pip:
## Pip installs can be listed next but should only be used when Conda is unavailable
#   - pmdarima==1.5.3


Overwriting /home/asruser/my-workspace/.recipes/recipe-Ru_oljS6g/requirements.txt


Search here for additional libraries https://anaconda.org/. This is the list of main **libraries already in use**:
`python=3.6.7` `scikit-learn` `pandas` `numpy` `data_access_sdk_python`
**Warning: libraries or specific versions you add may be incompatible with the above libraries**.

---

#### **Configuration Files**
List any hyperparameters you wish to use. Specify the dataset(s) and schema(s) that are needed for training/scoring. To find the dataset ids go to the **Data tab** in Adobe Experience Platform or view the **Datasets** folder in the **Notebooks Data tab** on the left. You can also find schema id in the **Notebooks Data tab** under the **Schemas** folder. Each configuration will only be used for its corresponding action. `ACP_DSW_TRAINING_XDM_SCHEMA` and `ACP_DSW_SCORING_RESULTS_XDM_SCHEMA` will only be used after the recipe has been created.

##### Training Configuration

In [20]:
%%writefile ~/my-workspace/.recipes/recipe-Ru_oljS6g/training.conf

{
   "trainingDataSetId": "6144b1cba3485119497a6f86",
   "criterion": "gini",
   "max_depth": 3
}

Overwriting /home/asruser/my-workspace/.recipes/recipe-Ru_oljS6g/training.conf


##### Scoring Configuration

In [21]:
%%writefile ~/my-workspace/.recipes/recipe-Ru_oljS6g/scoring.conf

{
   "scoringDataSetId": "6144b224fd59b719489bf07b",
   "scoringResultsDataSetId": "6148c1002e60e11949ac2ea3",
}

Overwriting /home/asruser/my-workspace/.recipes/recipe-Ru_oljS6g/scoring.conf


**The following configuration parameters are automatically set for you when you train/score:** 
`ML_FRAMEWORK_IMS_USER_CLIENT_ID` `ML_FRAMEWORK_IMS_TOKEN` `ML_FRAMEWORK_IMS_ML_TOKEN` `ML_FRAMEWORK_IMS_TENANT_ID`

---

#### **Training Data Loader File**
Implement the `load` function to load and prepare the training data.

In [22]:
%%writefile ~/my-workspace/.recipes/recipe-Ru_oljS6g/recipe/trainingdataloader.py

# LIBRARIES
import warnings
warnings.filterwarnings("ignore")
import numpy as np  
import pandas as pd      
from platform_sdk.dataset_reader import DatasetReader
from .utils import get_client_context
from sklearn.model_selection import train_test_split, KFold, cross_val_score

# FUNZIONE DI TRAINING
def load(config_properties):
    print("Training Data Load Start")

    #########################################
    # Load Data
    #########################################    
    client_context = get_client_context(config_properties)
    dataset_reader = DatasetReader(client_context, config_properties['trainingDataSetId'])
    titanic = dataset_reader.limit(10000000).read()
        
    #########################################
    # Data Preprocessing
    #########################################    
    titanic['Title'] = titanic.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
    new_titles = {
        "Capt":       "Officer",
        "Col":        "Officer",
        "Major":      "Officer",
        "Jonkheer":   "Royalty",
        "Don":        "Royalty",
        "Sir" :       "Royalty",
        "Dr":         "Officer",
        "Rev":        "Officer",
        "the Countess":"Royalty",
        "Dona":       "Royalty",
        "Mme":        "Mrs",
        "Mlle":       "Miss",
        "Ms":         "Mrs",
        "Mr" :        "Mr",
        "Mrs" :       "Mrs",
        "Miss" :      "Miss",
        "Master" :    "Master",
        "Lady" :      "Royalty"
    }

    titanic.Title = titanic.Title.map(new_titles)

    titanic.Cabin = titanic.Cabin.fillna(0)
    for i in range(len(titanic.Cabin)): 
        if titanic.Cabin[i] != 0:
            titanic.Cabin[i] = 1

    grouped = titanic.groupby(['Sex','Pclass', 'Title'])
    titanic.Age = grouped.Age.apply(lambda x: x.fillna(x.median()))

    most_embarked = titanic.Embarked.value_counts().index[0]
    titanic.Embarked = titanic.Embarked.fillna(most_embarked)
    titanic.Fare = titanic.Fare.fillna(titanic.Fare.median())

    titanic.drop('Name', axis =1, inplace=True)
    titanic.drop('Ticket', axis =1, inplace=True)
    titanic.drop('PassengerId', axis=1, inplace = True)
    
    titanic['SibSp'] = titanic['SibSp'].apply(lambda x: int(x))
    titanic['Parch'] = titanic['Parch'].apply(lambda x: int(x))
    
    Sex = {"male": 0, "female":1}
    titanic["Sex"] = titanic.Sex.map(Sex)
    titanic['Partner'] = titanic['SibSp'] + titanic['Parch'] # 
    titanic.drop(['SibSp', 'Parch'], axis=1, inplace=True)
    titanic = pd.get_dummies(titanic, columns = ['Title','Embarked'])
    
    #########################################
    # Return Dataset
    ######################################### 
    print('Return Dataset')
    print(titanic.shape)
    print(titanic.head())
    print("Training Data Load Finish")
    return titanic

Overwriting /home/asruser/my-workspace/.recipes/recipe-Ru_oljS6g/recipe/trainingdataloader.py


---

#### **Scoring Data Loader File**
Implement the `load` function to load and prepare the scoring data.

In [23]:
%%writefile ~/my-workspace/.recipes/recipe-Ru_oljS6g/recipe/scoringdataloader.py

# LIBRARIES
import warnings
warnings.filterwarnings("ignore")
import numpy as np  
import pandas as pd    
from platform_sdk.dataset_reader import DatasetReader
from .utils import get_client_context
from sklearn.model_selection import train_test_split, KFold, cross_val_score

def load(config_properties):

    print("Scoring Data Load Start")

    #########################################
    # Load Data
    #########################################    
    client_context = get_client_context(config_properties)
    dataset_reader = DatasetReader(client_context, config_properties['scoringDataSetId'])
    titanic = dataset_reader.limit(10000000).read()
    
    #########################################
    # Data Preprocessing
    #########################################    
    titanic['Title'] = titanic.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
    new_titles = {
        "Capt":       "Officer",
        "Col":        "Officer",
        "Major":      "Officer",
        "Jonkheer":   "Royalty",
        "Don":        "Royalty",
        "Sir" :       "Royalty",
        "Dr":         "Officer",
        "Rev":        "Officer",
        "the Countess":"Royalty",
        "Dona":       "Royalty",
        "Mme":        "Mrs",
        "Mlle":       "Miss",
        "Ms":         "Mrs",
        "Mr" :        "Mr",
        "Mrs" :       "Mrs",
        "Miss" :      "Miss",
        "Master" :    "Master",
        "Lady" :      "Royalty"
    }

    titanic.Title = titanic.Title.map(new_titles)

    titanic.Cabin = titanic.Cabin.fillna(0)
    for i in range(len(titanic.Cabin)): 
        if titanic.Cabin[i] != 0:
            titanic.Cabin[i] = 1

    grouped = titanic.groupby(['Sex','Pclass', 'Title'])
    titanic.Age = grouped.Age.apply(lambda x: x.fillna(x.median()))

    most_embarked = titanic.Embarked.value_counts().index[0]
    titanic.Embarked = titanic.Embarked.fillna(most_embarked)
    titanic.Fare = titanic.Fare.fillna(titanic.Fare.median())

    titanic.drop('Name', axis =1, inplace=True)
    titanic.drop('Ticket', axis =1, inplace=True)
    titanic.drop('PassengerId', axis=1, inplace = True)
    
    titanic['SibSp'] = titanic['SibSp'].apply(lambda x: int(x))
    titanic['Parch'] = titanic['Parch'].apply(lambda x: int(x))
    
    Sex = {"male": 0, "female":1}
    titanic["Sex"] = titanic.Sex.map(Sex)
    titanic['Partner'] = titanic['SibSp'] + titanic['Parch'] # 
    titanic.drop(['SibSp', 'Parch'], axis=1, inplace=True)
    titanic = pd.get_dummies(titanic, columns = ['Title','Embarked'])
    
    
    #########################################
    # Return Dataset
    #########################################
    print('Return Dataset')
    print(titanic.shape)
    print(titanic.head())
    print("Scoring Data Load Finish")
    return titanic

Overwriting /home/asruser/my-workspace/.recipes/recipe-Ru_oljS6g/recipe/scoringdataloader.py


---

#### **Pipeline File**
Implement the `train` function and return the trained model. Implement the `score` function to return a prediction made on the scoring data.

In [24]:
%%writefile ~/my-workspace/.recipes/recipe-Ru_oljS6g/recipe/pipeline.py

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, TimeSeriesSplit, RepeatedKFold, train_test_split
from sklearn.metrics import classification_report, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree

def train(config_properties, data):

    print("Train Start")

    #########################################
    # Define & Fit model
    #########################################
    y_train = data.Survived
    X_train = data.drop(columns = ['Survived'])
    print(X_train.shape)
    print(y_train.shape)
    model = DecisionTreeClassifier(criterion=config_properties['criterion'],
                                    max_depth=config_properties['max_depth'])
    model = model.fit(X_train, y_train)
    
    print("Train Complete")
    return model

def score(config_properties, data, model):

    print("Score Start")
    y_pred = model.predict(data)
    data['prediction'] = y_pred
    data = data.reset_index()

    print("Score Complete")
    return data

Overwriting /home/asruser/my-workspace/.recipes/recipe-Ru_oljS6g/recipe/pipeline.py


---

#### **Evaluator File**
Implement the `split` function to partition the training data and the `evaluate` function to the return the validation metrics you wish to see. Training output will be shown below this file cell.

In [25]:
%%writefile ~/my-workspace/.recipes/recipe-Ru_oljS6g/recipe/evaluator.py

from ml.runtime.python.core.regressionEvaluator import RegressionEvaluator
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split

class Evaluator(RegressionEvaluator):
    def __init__(self):
        print ("Initiate Evaluation")

    def split(self, config={}, dataframe=None):
        df_train, df_val = train_test_split(dataframe, test_size=0.20, random_state=16)
        return df_train, df_val

    def evaluate(self, data=[], model={}, config={}):
        print ("Evaluation evaluate triggered")
        val = data.drop('Survived', axis=1)
        y_pred = model.predict(val)
        y_actual = data['Survived'].values
        print(type(y_pred))
        print(type(y_actual))
        acc = accuracy_score(y_actual, y_pred)
        prec = precision_score(y_actual, y_pred, pos_label='1')
        metric = [{"name": "ACC", "value": acc, "valueType": "double"},
                 {"name": "PREC", "value": prec, "valueType": "double"}]
        
        print(metric)
        return metric

Overwriting /home/asruser/my-workspace/.recipes/recipe-Ru_oljS6g/recipe/evaluator.py


In [9]:
!bash -e run_action.sh recipe-Ru_oljS6g training

RUNNING TRAINING...
INSTALLING DEPENDENCIES...
Ignore the exception for python runtime cannot import name 'BlockBlobService' from 'azure.storage.blob' (/opt/conda/envs/usermlruntimepython/lib/python3.7/site-packages/azure/storage/blob/__init__.py)
2021-09-21 08:59:28,756 INFO trainingInitiator.py: Trainer initiated
INFO:trainingInitiator.py:Trainer initiated
2021-09-21 08:59:28,767 INFO trainingInitiator.py: Evaluator initiated
INFO:trainingInitiator.py:Evaluator initiated
2021-09-21 08:59:28,768 INFO main.py: Training starts, testing:True, conf:/tmp/tmp.EqQEpT5PxZ/token.conf
INFO:main.py:Training starts, testing:True, conf:/tmp/tmp.EqQEpT5PxZ/token.conf
2021-09-21 08:59:28,769 INFO trainingInitiator.py: Training class is not of type Tensorflow
INFO:trainingInitiator.py:Training class is not of type Tensorflow
2021-09-21 08:59:28,769 INFO trainingInitiator.py: Python Job
INFO:trainingInitiator.py:Python Job
2021-09-21 08:59:28,769 INFO trainingInitiator.py: Load the dataframe
INFO:trai

---

#### **Data Saver File**
Implement the `save` function for saving your prediction. Scoring output will be added below this cell.

In [26]:
%%writefile ~/my-workspace/.recipes/recipe-Ru_oljS6g/recipe/datasaver.py

import pandas as pd
from .utils import get_client_context
from platform_sdk.models import Dataset
from platform_sdk.dataset_writer import DatasetWriter
from datetime import date

def save(config_properties, prediction):
    print("Datasaver Start")
    print(prediction)
    prediction['timestamp'] = pd.to_datetime(date.today())
    prediction = prediction[['prediction', 'Sex', 'timestamp']]

    prediction.rename({'prediction':'output.PREDICTION',
                     'Sex':'output.SEX',
                     'timestamp':'output.TIMESTAMP'
                    },
                      inplace = True, axis = 1)

    client_context = get_client_context(config_properties)
    tenant_id = config_properties.get("tenantId")
    prediction = prediction.add_prefix(tenant_id+".")
    print(prediction.columns)
    print('prediction after renaming: ', prediction.head(5))
    dataset = Dataset(client_context).get_by_id(config_properties['scoringResultsDataSetId'])
    print(dataset)
    dataset_writer = DatasetWriter(client_context, dataset)
    dataset_writer.write(prediction, file_format='json')

    print("Datasaver Finish")
    print(prediction)

Overwriting /home/asruser/my-workspace/.recipes/recipe-Ru_oljS6g/recipe/datasaver.py


In [18]:
!bash -e run_action.sh recipe-Ru_oljS6g scoring

RUNNING SCORING...
Ignore the exception for python runtime cannot import name 'BlockBlobService' from 'azure.storage.blob' (/opt/conda/envs/usermlruntimepython/lib/python3.7/site-packages/azure/storage/blob/__init__.py)
2021-09-21 09:03:53,262 INFO main.py: Scoring starts, testing:True, conf:/tmp/tmp.WWQaLyPfZl/token.conf
2021-09-21 09:03:56,368 INFO scoringInitiator.py: Scorer initiated
INFO:scoringInitiator.py:Scorer initiated
2021-09-21 09:03:56,372 INFO scoringInitiator.py: Scoring class is not of type Tensorflow
INFO:scoringInitiator.py:Scoring class is not of type Tensorflow
2021-09-21 09:03:56,372 INFO scoringInitiator.py: Python scoring starts
INFO:scoringInitiator.py:Python scoring starts
2021-09-21 09:03:56,373 INFO scoringInitiator.py: Python executed scoring
INFO:scoringInitiator.py:Python executed scoring
Scoring Data Load Start
INFO:PlatformSDKPython:dataset_reader: seconds taken to get dataset details from catalog and make PQS connection: 1.17
INFO:PlatformSDKPython:data